# Topic Modeling

In [2]:
import pandas as pd
import re
import numpy as np
import PyPDF2
import requests
import io
from PyPDF2 import PdfFileReader

In [19]:
url = 'https://www.nkeconwatch.com/nk-uploads/KJU-speech-2013.pdf'


'''convert PDF to BytesIO file'''
r = requests.get(url)
f = io.BytesIO(r.content)


'''extracting PDF text'''
reader = PdfFileReader(f)
number_of_pages = reader.getNumPages()

content = []
for page_number in range(number_of_pages):
    page = reader.getPage(page_number)
    content += page.extractText().split('\n')

    
'''convert list to string'''
def listToString(s):
    str1 = ""
    for ele in s:
        str1 += ele
    
    return str1

speech = listToString(content)


'''split text into individual sentences'''
text = speech.split('. ')

In [20]:
df = pd.DataFrame(text, columns = ['text'])
df

,text
0,New Year Address Made by Kim Jong Un Pyongyang...
1,"The following is its full text: Dear comrades,..."
2,Reflecting the unanimous reverence of all the ...
3,I extend my warm greetings to the service pers...
4,My New Year greetings also go to our compatrio...
...,...
110,The present international situation demands th...
111,"By holding fast to the ideals of independence,..."
112,A brighter and broader vista is open for us in...
113,Let us vigorously strive to achieve the prospe...


In [22]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

stopwords = stopwords.words('english')
punctuation = '!"$%&\'()*+,-./:;<=>?[\\]^_;{|}'

def clean_text(text):
    '''lowercase text'''
    text = text.lower()
    '''remove puncutation'''
    text = re.sub('['+punctuation+']+', ' ', text)
    '''remove numbers'''
    text = re.sub('([0-9]+)', '', text)
    
    return text

df['clean_text'] = df.text.apply(clean_text)
df

,text,clean_text
0,New Year Address Made by Kim Jong Un Pyongyang...,new year address made by kim jong un pyongyang...
1,"The following is its full text: Dear comrades,...",the following is its full text dear comrades ...
2,Reflecting the unanimous reverence of all the ...,reflecting the unanimous reverence of all the ...
3,I extend my warm greetings to the service pers...,i extend my warm greetings to the service pers...
4,My New Year greetings also go to our compatrio...,my new year greetings also go to our compatrio...
...,...,...
110,The present international situation demands th...,the present international situation demands th...
111,"By holding fast to the ideals of independence,...",by holding fast to the ideals of independence ...
112,A brighter and broader vista is open for us in...,a brighter and broader vista is open for us in...
113,Let us vigorously strive to achieve the prospe...,let us vigorously strive to achieve the prospe...


In [24]:
def token_text(text):
    '''remove stopwords'''
    text_token_list = [word for word in text.split(' ')
                       if word not in stopwords]
    text = ' '.join(text_token_list)
    
    return text

df['token_text'] = df.clean_text.apply(token_text)
df

,text,clean_text,token_text
0,New Year Address Made by Kim Jong Un Pyongyang...,new year address made by kim jong un pyongyang...,new year address made kim jong un pyongyang j...
1,"The following is its full text: Dear comrades,...",the following is its full text dear comrades ...,following full text dear comrades officers m...
2,Reflecting the unanimous reverence of all the ...,reflecting the unanimous reverence of all the ...,reflecting unanimous reverence servi ce person...
3,I extend my warm greetings to the service pers...,i extend my warm greetings to the service pers...,extend warm greetings service personnel peopl...
4,My New Year greetings also go to our compatrio...,my new year greetings also go to our compatrio...,new year greetings also go compatriots south ...
...,...,...,...
110,The present international situation demands th...,the present international situation demands th...,present international situation demands ou r r...
111,"By holding fast to the ideals of independence,...",by holding fast to the ideals of independence ...,holding fast ideals independence peac e frien...
112,A brighter and broader vista is open for us in...,a brighter and broader vista is open for us in...,brighter broader vista open us new year victo...
113,Let us vigorously strive to achieve the prospe...,let us vigorously strive to achieve the prospe...,let us vigorously strive achieve prosperity co...


In [49]:
x = df["token_text"].apply(str)

In [51]:
x.

0      new year address made kim jong un pyongyang  j...
1      following full text  dear comrades  officers m...
2      reflecting unanimous reverence servi ce person...
3      extend warm greetings service personnel  peopl...
4      new year greetings also go compatriots  south ...
                             ...                        
110    present international situation demands ou r r...
111    holding fast ideals independence  peac e frien...
112    brighter broader vista open us new year  victo...
113    let us vigorously strive achieve prosperity co...
114                                                     
Name: token_text, Length: 115, dtype: object

In [50]:
import spacy 

nlp = spacy.load('en_core_web_sm')

doc = nlp(x)

# Extract the lemma for each token and join
df['lemma_token_text'] = " ".join([token.lemma_ for token in doc])

ValueError: [E1041] Expected a string, Doc, or bytes as input, but got: <class 'pandas.core.series.Series'>